**برای اجرای کد به انتهای فایل بروید**

In [ ]:
import random
from typing import Callable, Dict, List, Tuple, TypeVar
import sys
sys.path.insert(1, r"sentiment-main")
from util import *

In [ ]:
FeatureVector = Dict[str, int]
WeightVector = Dict[str, float]
Example = Tuple[FeatureVector, int]

### Problem 3: binary classification

##### Problem 3a: feature extraction

In [ ]:
def extractWordFeatures(x: str) -> FeatureVector:
    """
    Extract word features for a string x. Words are delimited by
    whitespace characters only.
    @param string x:
    @return dict: feature vector representation of x.
    Example: "I am what I am" --> {'I': 2, 'am': 2, 'what': 1}
    """
    # BEGIN_YOUR_CODE (our solution is 4 lines of code, but don't worry if you deviate from this)
    words = x.split()  # Split the string into a list of words
    featureVector  = {}  # Initialize an empty dictionary to store the feature vector
    for word in words:  # Iterate over each word in the list
        featureVector[word] = featureVector.get(word, 0) + 1  # Increment the count of the word in the feature vector
    return featureVector  # Return the feature vector
    # END_YOUR_CODE

##### Problem 3b: stochastic gradient descent

In [ ]:
def learnPredictor(trainExamples: List[Tuple[T, int]],
                   validationExamples: List[Tuple[T, int]],
                   featureExtractor: Callable[[T], FeatureVector],
                   numEpochs: int, eta: float) -> WeightVector:
    '''
    Given |trainExamples| and |validationExamples| (each one is a list of (x,y)
    pairs), a |featureExtractor| to apply to x, and the number of epochs to
    train |numEpochs|, the step size |eta|, return the weight vector (sparse
    feature vector) learned.

    You should implement stochastic gradient descent.

    Notes:
    - Only use the trainExamples for training!
    - You should call evaluatePredictor() on both trainExamples and validationExamples
    to see how you're doing as you learn after each epoch.
    - The predictor should output +1 if the score is precisely 0.
    '''
    weights = {}  # feature => weight

    # BEGIN_YOUR_CODE (our solution is 13 lines of code, but don't worry if you deviate from this)
    for epoch in range(numEpochs):
        for example, label in trainExamples:
            feature = featureExtractor(example)
            margin = dotProduct(weights, feature) * label
            if margin < 1:
                # Update the weights using stochastic gradient descent
                increment(weights, eta * label, feature)
        # Create a predictor function based on the current weights
        predictor = lambda example: 1 if dotProduct(featureExtractor(example), weights) >= 0 else -1
        # Evaluate the training error using the predictor function
        errorInTraining = evaluatePredictor(trainExamples, predictor)
        # Evaluate the validation error using the predictor function
        errorInValidation = evaluatePredictor(validationExamples, predictor)
        # Print the epoch number and the corresponding errors
        print(f"Epoch {epoch + 1}/{numEpochs}: Train Error = {errorInTraining:.2f}, Validation Error = {errorInValidation:.2f}")
    # END_YOUR_CODE

    return weights

##### Problem 3c: generate test case

In [27]:
def generateDataset(numExamples: int, weights: WeightVector) -> List[Example]:
    '''
    Return a set of examples (phi(x), y) randomly which are classified correctly by
    |weights|.
    '''
    random.seed(42)

    # Return a single example (phi(x), y).
    # phi(x) should be a dict whose keys are a subset of the keys in weights
    # and values can be anything (randomize!) with a score for the given weight vector.
    # note that there is intentionally flexibility in how you define phi.
    # y should be 1 or -1 as classified by the weight vector.
    # y should be 1 if the score is precisely 0.pytho

    # Note that the weight vector can be arbitrary during testing.
    def generateExample() -> Tuple[Dict[str, int], int]:
        # BEGIN_YOUR_CODE (our solution is 3 lines of code, but don't worry if you deviate from this)
        # Create an empty dictionary to store the feature vector phi
        phi = {feature: random.choice([0, 1]) for feature in weights.keys()}  # Randomly assign values (0 or 1) to each feature in phi
        y = 1 if sum(weights[f] * phi[f] for f in phi) >= 0 else -1  # Calculate the score for the given weight vector and classify y as 1 or -1
        # END_YOUR_CODE
        return phi, y

    return [generateExample() for _ in range(numExamples)]

##### Problem 3d: character features

In [28]:
def extractCharacterFeatures(n: int) -> Callable[[str], FeatureVector]:
    '''
    Return a function that takes a string |x| and returns a sparse feature
    vector consisting of all n-grams of |x| without spaces mapped to their n-gram counts.
    EXAMPLE: (n = 3) "I like tacos" --> {'Ili': 1, 'lik': 1, 'ike': 1, ...
    You may assume that n >= 1.
    '''
    def extract(x: str) -> Dict[str, int]:
        # BEGIN_YOUR_CODE (our solution is 6 lines of code, but don't worry if you deviate from this)
        # Remove spaces from the input string and convert it to lowercase
        x_no_spaces = x.replace(' ', '').lower()
        # Initialize an empty dictionary to store the character sequences and their counts
        character_sequence = {}
        # Iterate over each substring of length n in the input string
        for i in range(len(x_no_spaces) - n + 1):
            # Extract the current substring
            substring = x_no_spaces[i:i+n]
            # Check if the substring is already in the dictionary
            if substring in character_sequence:
                # If it is, increment its count by 1
                character_sequence[substring] = character_sequence[substring] + 1
            else:
                # If it is not, add it to the dictionary with a count of 1
                character_sequence[substring] = 1
        # Return the dictionary containing the character sequences and their counts
        return character_sequence
        # END_YOUR_CODE

    return extract

##### Problem 3e:

In [ ]:
def testValuesOfN(n: int):
    '''
    Use this code to test different values of n for extractCharacterFeatures
    This code is exclusively for testing.
    Your full written solution for this problem must be in sentiment.pdf.
    '''
    trainExamples = readExamples(r"polarity.train")
    validationExamples = readExamples(r"polarity.train")
    featureExtractor = extractCharacterFeatures(n)
    weights = learnPredictor(trainExamples,
                             validationExamples,
                             featureExtractor,
                             numEpochs=20,
                             eta=0.01)
    outputWeights(weights, 'weights')
    outputErrorAnalysis(validationExamples, featureExtractor, weights,
                        'error-analysis')  # Use this to debug
    trainError = evaluatePredictor(
        trainExamples, lambda x:
        (1 if dotProduct(featureExtractor(x), weights) >= 0 else -1))
    validationError = evaluatePredictor(
        validationExamples, lambda x:
        (1 if dotProduct(featureExtractor(x), weights) >= 0 else -1))
    print(("Official: train error = %s, validation error = %s" %
           (trainError, validationError)))

**Question:**
Experiment with different values of n to see which one produces the smallest validation error. You should observe that this error is nearly as small as that produced by word features. Why is this the case?

**Answer:**
The reason why character features (n-grams) can produce nearly as small validation error as word features is because n-grams can capture the local structure of the language. Imagine a 3-gram can capture the structure of a three-letter word or a three-word phrase. This local structure can be very informative for many tasks, such as text classification or sentiment analysis.
For example: "The food was not good, it was fantastic!"
In this case, character n-grams might outperform word features because they can capture the negation "not good" and the contrast indicated by "it was fantastic". Word features might treat "not", "good", and "fantastic" as independent features and miss the overall sentiment of the sentence.

##### Test Functions

In [10]:
!python ./grader.py

========== START GRADING
----- START PART 3a-0-basic: basic test
----- END PART 3a-0-basic [took 0:00:00 (max allowed 1 seconds), 1/1 points]

----- START PART 3a-1-hidden: test multiple instances of the same word in a sentence
----- END PART 3a-1-hidden [took 0:00:00 (max allowed 1 seconds), ???/1 points (hidden test ungraded)]

----- START PART 3b-0-basic: basic sanity check for learning correct weights on two training and testing examples each
Epoch 1/20: Train Error = 0.00, Validation Error = 0.00
Epoch 2/20: Train Error = 0.00, Validation Error = 0.00
Epoch 3/20: Train Error = 0.00, Validation Error = 0.00
Epoch 4/20: Train Error = 0.00, Validation Error = 0.00
Epoch 5/20: Train Error = 0.00, Validation Error = 0.00
Epoch 6/20: Train Error = 0.00, Validation Error = 0.00
Epoch 7/20: Train Error = 0.00, Validation Error = 0.00
Epoch 8/20: Train Error = 0.00, Validation Error = 0.00
Epoch 9/20: Train Error = 0.00, Validation Error = 0.00
Epoch 10/20: Train Error = 0.00, Validation E